In [1]:
import os
from pathlib import Path

import pandas as pd

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True
pd.options.plotting.backend = "plotly"


current_dir = Path(os.getcwd())

PROJECT = current_dir.parent.resolve()
SRC = PROJECT.parent.resolve()
ROOT = PROJECT.joinpath("..", "..").resolve()

BLD = ROOT.joinpath("bld").resolve()

# Folders inside idos_ppp
DATA = PROJECT.joinpath("data").resolve()
DATA_MGT = PROJECT.joinpath("data_management").resolve()
ANALYSIS = PROJECT.joinpath("analysis").resolve()
FINAL = PROJECT.joinpath("final").resolve()

DOCUMENTS = ROOT.joinpath("documents").resolve()

print(ROOT)

/Users/sergeimolinari/Desktop/IDOS/IDOS-PPP


# TRY THE NEW DATASET FOR POSSIBLE ERRORS

In [5]:
from idos_ppp.config import DATA
from idos_ppp.parameters import sheet_names

data_file = DATA
raw_data_path=DATA / "Data_2007_2010_2013_2016_2019.xlsx"

# Open the dataset file using pandas
try:
    raw_dta = {
        sheet: pd.read_excel(raw_data_path, sheet_name=sheet, header=1)
        for sheet in sheet_names
    }
    print("Data loaded successfully.")
    # Display the first few rows of one of the dataframes
    for sheet, df in raw_dta.items():
        print(f"First few rows of {sheet}:")
        print(df.head())
except FileNotFoundError:
    print(f"Error: The file {raw_data_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Data loaded successfully.
First few rows of Transformed 2007:
    Unnamed: 0   Unnamed: 1 FFP X1   1-2 UCDP WEF GCI Subpillar: 5. Security  \
0  Afghanistan  Afghanistan       0         0                            XXX   
1      Albania      Albania    0.44         1                       0.691667   
2      Algeria      Algeria    0.43  0.458492                       0.768333   
3       Angola       Angola    0.24  0.899611                           0.86   
4    Argentina    Argentina    0.72         1                           0.69   

  1-4 Political Terror 1-5 FFP P3 Protection  \
0                    0       0.18      0.045   
1                 0.75       0.46   0.668333   
2                 0.25       0.26       0.43   
3                  0.5       0.25        0.5   
4                 0.75       0.63       0.72   

  WEF GCR 2nd pillar: Infrastructure 2-1-2 UN: Telecom (2008)   ...  \
0                                XXX                   0.01576  ...   
1                         

In [ ]:
def clean_and_concatenate(
    raw_data=DATA / "Data_2007_2010_2013_2016_2019.xlsx",
    produces=BLD / "data" / "clean_data_new.csv",
):
    raw = {
        year: pd.read_excel(raw_data, sheet_name=sheet, header=1)
        for year, sheet in zip(years, sheet_names, strict=False)
    }
    countries_list = pd.read_excel(raw_data, sheet_name="3-2", header=0)
    clean_data = clean_and_concatenate_data(
        raw_data_dict=raw, country_codes=countries_list.iloc[0:, 1].tolist()
    )
    clean_data.to_csv(produces)

In [9]:
from idos_ppp.config import DATA, BLD
from idos_ppp.data_management.idos_datamanagement import clean_and_concatenate_data
from idos_ppp.parameters import missing_countries, sheet_names, years

bld_file = BLD
    
# Construct the path to the files
raw_data_path=DATA / "Data_2007_2010_2013_2016_2019.xlsx"
clean_data_path = BLD / "data" / "clean_data_new.csv"

try:
    clean_data = clean_and_concatenate_data(
    raw_data_dict={
        year: pd.read_excel(raw_data_path, sheet_name=sheet, header=1)
        for year, sheet in zip(years, sheet_names, strict=False)
    },
    country_codes=pd.read_excel(raw_data_path, sheet_name="3-2", header=0).iloc[0:, 1].tolist()
    )
    clean_data.to_csv(clean_data_path)
    print("Data loaded successfully.")
    # Display the first few rows of the dataframe
    print(clean_data.head())
except FileNotFoundError:
    print(f"Error: The file {clean_data_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Data loaded successfully.
               country_name  year  1_1_ffp_x1  1_2_ucdp  \
country_alpha3                                            
AFG             Afghanistan  2007         0.0       0.0   
ALB                 Albania  2007        0.44       1.0   
DZA                 Algeria  2007        0.43  0.458492   
AGO                  Angola  2007        0.24  0.899611   
ARG               Argentina  2007        0.72       1.0   

                1_3_wef_gci_subpillar:_5._security  1_4_political_terror  \
country_alpha3                                                             
AFG                                           <NA>                   0.0   
ALB                                       0.691667                  0.75   
DZA                                       0.768333                  0.25   
AGO                                           0.86                   0.5   
ARG                                           0.69                  0.75   

                1_5_ffp_p3 

In [ ]:
clean_data

# PLAY WITH LISTS OF COUNTRIES

In [10]:
european_union = [
    "Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czechia", "Denmark",
    "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland",
    "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland",
    "Portugal", "Romania", "Slovak Republic", "Slovenia", "Spain", "Sweden"
]
len(european_union)

27

In [11]:
# Conflic / Post-conflict (from Heydemann 2025)
conflict_countries = [
    "Libya", "Syrian Arab Republic", "Yemen, Rep."
]

conflict_and_postconflict = [
    "Iraq", "Lebanon", "Libya", "Syrian Arab Republic", "Yemen, Rep."
]

# GCC countries (and repressive ones) (from Heydemann 2025)
gcc_high_income_countries = [
    "Bahrain", "Kuwait", "Oman", "Qatar", "Saudi Arabia", "United Arab Emirates"
]

repressive_countries = [
    "Egypt, Arab Rep.", "Jordan", "Morocco", "Tunisia"
] # Tunisia post-2020

gcc_and_repressive_countries = [
    "Bahrain", "Kuwait", "Oman", "Qatar", "Saudi Arabia", "United Arab Emirates", "Egypt, Arab Rep.", "Jordan", "Morocco", "Tunisia"
] # Tunisia post-2020

print(len(conflict_countries), len(conflict_and_postconflict), len(gcc_high_income_countries), len(repressive_countries), len(gcc_and_repressive_countries))

3 5 6 4 10
